# Mod4 Project

## Import Libraries

In [21]:

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from collections import defaultdict


# pd.set_option("display.max_rows", None, "display.max_columns", None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Import data files
movies_df = pd.read_csv('movies.csv',index_col=False)
ratings_df = pd.read_csv('ratings.csv',index_col=False)
tags_df = pd.read_csv('tags.csv',index_col=False)

First thing is to look at each file to get an idea of what is contained. Then we will merge any relavent data sets and perform some EDA

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


We're going to drop the timestamp column as it is not useful for our model

In [5]:
ratings_df.drop(columns='timestamp',inplace=True)

In [6]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:

movies_df.set_index('movieId')

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [8]:
df = pd.merge(movies_df,tags_df, on='movieId', how='left')

In [9]:
tags_df.shape

(3683, 4)

In [10]:
ratings_df.shape

(100836, 3)

In [11]:
df.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
3,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
4,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [12]:
df.shape

(11853, 6)

## EDA

## Collaborative Model

The first thing we are going to do is build some baseline Collaborative Recommendation models. We will begin with Surprise and loading in our data. We'll first look at RMSE for evaluation but then expand to other metrics.

In [13]:

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [14]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


In [15]:
#Creating SVD Model and showing RMSE
params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)
print(g_s_svd.best_score)
print(g_s_svd.best_params)

{'rmse': 0.8692191787250529, 'mae': 0.6686653944430218}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [16]:
sim_metrics = ['pearson','MSD','cosine','pearson_baseline']
#Creating Basic KNN Model and showing RMSE, checking all distance metrics
for metric in sim_metrics:
    knn_basic = KNNBasic(sim_options={'name': metric, 'user_based':True})
    cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)
    print('Similarity Metric = ', metric,'---', 'test_rmse = ', np.mean(cv_knn_basic['test_rmse']),'\n')
    
    

Similarity Metric =  pearson --- test_rmse =  0.9717069919558838 

Similarity Metric =  MSD --- test_rmse =  0.9476963794471122 

Similarity Metric =  cosine --- test_rmse =  0.9715822007047269 

Similarity Metric =  pearson_baseline --- test_rmse =  0.9733539568151228 



In [17]:
# cross validating with KNNBaseline
for metric in sim_metrics:
    knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
    cv_knn_baseline = cross_validate(knn_baseline,data)
    print('Similarity Metric = ', metric,'---', 'test_rmse = ', np.mean(cv_knn_baseline['test_rmse']),'\n')

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Similarity Metric =  pearson --- test_rmse =  0.8762871715023689 

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matri

### Trying hit rate as an evaluation

In [18]:
trainSet, testSet = train_test_split(data, test_size=.25, random_state=0)

In [27]:
#Get top 10 predictions for ratings above 4
def GetTopN(predictions, n=10, minimumRating=4.0):
    topN = defaultdict(list)
    for userID, movieID, actualRating, estimatedRating, _ in predictions:
        if (estimatedRating >= minimumRating):
            topN[int(userID)].append((int(movieID), estimatedRating))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN
    
LOOCV = LeaveOneOut(n_splits=1, random_state=1)



In [50]:
#Get top predictions 
#When I use n=10 for knn it gets 0 hits, expanding to 50 it got a positive non-zero amount
#so I am confident it is working just doing a worse job likely due to sparseness. I expanded
#both to 50 for now
svd_model= SVD(n_factors=50,reg_all=0.05)
KNN_model= KNNBaseline(sim_options={'name':'pearson', 'user_based':True})

for trainSet, testSet in LOOCV.split(data):
    # Train model without left-out ratings
    svd_model.fit(trainSet)
    KNN_model.fit(trainSet)
    # Predicts ratings for left-out ratings only
    leftOutPredictionsSVD = svd_model.test(testSet)
    leftOutPredictionsKNN = KNN_model.test(testSet)
    # Build predictions for all ratings not in the training set
    bigTestSet = trainSet.build_anti_testset()
    allPredictionsSVD = svd_model.test(bigTestSet)
    bigTestSet = trainSet.build_anti_testset()
    allPredictionsKNN = KNN_model.test(bigTestSet)
    # Compute top 10 recs for each user
    topNPredictedSVD = GetTopN(allPredictionsSVD, n=50)
    topNPredictedKNN = GetTopN(allPredictionsKNN, n=50)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [31]:
def HitRate(topNPredicted, leftOutPredictions):
    hits = 0
    total = 0

 # For each left-out rating
    for leftOut in leftOutPredictions:
        userID = leftOut[0]
        leftOutMovieID = leftOut[1]
        # Is it in the predicted top 10 for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == int(movieID)):
                hit = True
                break
        if (hit) :
            hits += 1

        total += 1

    # Compute overall precision
    return hits/total


In [52]:
print("\nHit Rate SVD: ", HitRate(topNPredictedSVD, leftOutPredictionsSVD))
print("\nHit Rate KNN: ", HitRate(topNPredictedKNN, leftOutPredictionsKNN))


Hit Rate SVD:  0.07540983606557378

Hit Rate KNN:  0.003278688524590164


In [33]:
def RatingHitRate(topNPredicted, leftOutPredictions):
    hits = defaultdict(float)
    total = defaultdict(float)
    # For each left-out rating
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        # Is it in the predicted top N for this user?
        hit = False
        for movieID, predictedRating in topNPredicted[int(userID)]:
            if (int(leftOutMovieID) == movieID):
                hit = True
                break
        if (hit) :
            hits[actualRating] += 1
        total[actualRating] += 1

    # Compute overall precision
    for rating in sorted(hits.keys()):
        print(rating, hits[rating] / total[rating])

In [53]:
print("Hit Rate by Rating value SVD: ")
RatingHitRate(topNPredictedSVD, leftOutPredictionsSVD)

Hit Rate by Rating value SVD: 
2.5 0.06666666666666667
3.0 0.017391304347826087
3.5 0.020833333333333332
4.0 0.07777777777777778
4.5 0.22641509433962265
5.0 0.13008130081300814


In [54]:
print("Hit Rate by Rating value KNN: ")
RatingHitRate(topNPredictedKNN, leftOutPredictionsKNN)

Hit Rate by Rating value KNN: 
4.0 0.005555555555555556
5.0 0.008130081300813009


In [55]:
def CumulativeHitRate(topNPredicted, leftOutPredictions, ratingCutoff=0):
    hits = 0
    total = 0
    # For each left-out rating
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        # Only look at ability to recommend things the users actually liked...
        if (actualRating >= ratingCutoff):
            # Is it in the predicted top 10 for this user?
            hit = False
            for movieID, predictedRating in topNPredicted[int(userID)]:
                if (int(leftOutMovieID) == movieID):
                    hit = True
                    break
            if (hit) :
                hits += 1
            total += 1

        # Compute overall precision
    return hits/total

In [56]:
print("Cumulative Hit Rate SVD (rating >= 4): ", CumulativeHitRate(topNPredictedSVD, leftOutPredictionsSVD, 4.0))



Cumulative Hit Rate SVD (rating >= 4):  0.11797752808988764


In [57]:
print("Cumulative Hit Rate KNN(rating >= 4): ", CumulativeHitRate(topNPredictedKNN, leftOutPredictionsKNN, 4.0))


Cumulative Hit Rate KNN(rating >= 4):  0.0056179775280898875


In [58]:
def AverageReciprocalHitRank(topNPredicted, leftOutPredictions):
    summation = 0
    total = 0
        # For each left-out rating
    for userID, leftOutMovieID, actualRating, estimatedRating, _ in leftOutPredictions:
        # Is it in the predicted top N for this user?
        hitRank = 0
        rank = 0
        for movieID, predictedRating in topNPredicted[int(userID)]:
            rank = rank + 1
            if (int(leftOutMovieID) == movieID):
                hitRank = rank
                break
        if (hitRank > 0) :
                summation += 1.0 / hitRank

        total += 1

    return summation / total

In [61]:
print("Average Reciprocal Hit Rank SVD: ", AverageReciprocalHitRank(topNPredictedSVD, leftOutPredictionsKNN))


Average Reciprocal Hit Rank SVD:  0.012580102122874436


In [62]:
print("Average Reciprocal Hit Rank KNN: ", AverageReciprocalHitRank(topNPredictedKNN, leftOutPredictionsKNN))


Average Reciprocal Hit Rank KNN:  0.00017041000647558026
